In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'creditcardfraud:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F310%2F23498%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240730%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240730T110945Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D157947d38fda9a664ab7be89424ff0d17f296eef1b648850251a399459f10e61d0d1fb86bf92a2cdab37386136580553aa897f5c1e9495d1d85db1a92f5c13c46dbb00a12792c6b02762b623691ae9bb0377e0622b0c20e8928fd92301d7eab8b0d5d780c4335787650b1a33a95bf1d1b4df45f8af379929a1259057e033552b6e435eafdf32c250aef04b9d813a850e72a195798a401bf4797c477b8b2301488bfd8cd5f8fed7ba359c3bab2de5367753807f7566b4af85a155dd6b4c2faa2794be33c1292fc721e8a69e62d355be62e5acb8b81dbb9e695e445d4b85f98a5837cdc0570cbcc565552fb7cd2d1f923ce69922fd591e54fb6450e7a5cad2c4e3'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import confusion_matrix
import seaborn as sns
from imblearn.over_sampling import SMOTE
from scipy.stats import zscore
from sklearn.ensemble import RandomForestClassifier

In [ ]:
df = pd.read_csv("/kaggle/input/creditcardfraud/creditcard.csv")

In [ ]:
df["Class"].value_counts()

In [ ]:
df.duplicated().sum()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df['Class_Jittered'] = df['Class'] + np.random.normal(0, 0.1, size=len(df))

plt.figure(figsize=(10, 6))
plt.scatter(df['Amount'], df['Class_Jittered'], c=df['Class'], cmap='bwr', alpha=0.7, edgecolors='w')

plt.xlabel('Amount $|£|€')
plt.ylabel('Class')
plt.yticks([0, 1], ['Real', 'Fake'])
plt.title('Scatter Plot of Transaction Amounts with Jitter')
plt.colorbar(label='Class')
plt.grid(True)

plt.show()

In [ ]:
threshold = 3
df['zscores'] = zscore(df['Amount'])
df_filtered = df[(df['zscores'] < threshold) & (df['zscores'] > -threshold)]
df = df_filtered.drop(columns=['zscores'])

In [ ]:
df['Class_Jittered'] = df['Class'] + np.random.normal(0, 0.1, size=len(df))

plt.figure(figsize=(10, 6))
plt.scatter(df['Time'], df['Class_Jittered'], c=df['Class'], cmap='bwr', alpha=0.7, edgecolors='w')

plt.xlabel('Time')
plt.ylabel('Class')
plt.yticks([0, 1], ['Real', 'Fake'])
plt.title('Scatter Plot of Transaction Time with Jitter')
plt.colorbar(label='Class')
plt.grid(True)

plt.show()

In [ ]:
def normalize_data(X):
    return (X - X.min()) / (X.max() - X.min())


In [ ]:

X_features = df.drop(columns=["Class"])
Y_label = df["Class"]

X_train, X_test, y_train, y_test = train_test_split(X_features, Y_label, test_size=0.2, random_state=42)
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_features, Y_label = smote.fit_resample(X_train, y_train)
X_features = normalize_data(X_features)
#X_features = normalise_data(X_features)
X_features_train, X_features_test, y_label_train, y_label_test = train_test_split(X_features, Y_label, train_size=.8, random_state=42)

In [ ]:
knn = KNeighborsClassifier(3, weights="distance", algorithm="auto")
knn.fit(X_features_train, y_label_train)

In [ ]:
print(len(X_features_train))
print(len(X_features_test))
print(X_features_test)

In [ ]:
predictions = knn.predict(X_features_test)

In [ ]:
scores = knn.score(X_features_test, y_label_test)

In [ ]:
scores

In [ ]:
prec_recall_fscore_supp = precision_recall_fscore_support(y_label_test, predictions, labels=df["Class"].unique())

In [ ]:
cm = confusion_matrix(y_label_test, predictions, labels=[1, 0])
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', xticklabels=['Predicted Fake', 'Predicted Real'], yticklabels=['Actual Fake', 'Actual Real'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix Heatmap')
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve
def plot_roc_curve(true_y, y_prob):
    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.title("Credit Card Fraudulent detection ROC Curve")
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.show()

https://www.w3schools.com/python/python_ml_auc_roc.asp

In [ ]:
plot_roc_curve(y_label_test, predictions)

In [ ]:
rfc = RandomForestClassifier(n_estimators=300, max_depth=10, max_features="log2", bootstrap=True, random_state=42)
rfc.fit(X_features_train, y_label_train)

In [ ]:
rfc_pred = rfc.predict(X_features_test)

In [ ]:
plot_roc_curve(y_label_test, rfc_pred)

In [ ]:
"""rfc.score(X_features_test, y_label_test)"""

In [ ]:
"""from sklearn.model_selection import cross_val_score
import numpy as np
cv_scores = cross_val_score(rfc, X_features_train, y_label_train, cv=5, scoring='accuracy')
print(f"Cross-Validation Accuracy Scores: {cv_scores}")
print(f"Mean Accuracy: {np.mean(cv_scores)}")
"""

In [ ]:
"""cm = confusion_matrix(y_label_test, rfc_pred, labels=[1, 0])
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Reds', xticklabels=['Predicted Fake', 'Predicted Real'], yticklabels=['Actual Fake', 'Actual Real'])

# Customize the plot
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix Heatmap')
plt.show()
"""